# Data exploration and pre-process:-

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Dataset/Supply_Chain_Dataset_New.csv', encoding='latin1', engine='python')


Mounted at /content/drive


In [2]:
#drop unwanted columns
unwanted_columns =["Customer Email","Customer Zipcode","Customer Street","Customer Fname","Customer Id","Customer Lname","Customer Segment","Latitude","Longitude","Order Customer Id","order date (DateOrders)","Order Id","Order Item Cardprod Id","Order Item Id","Order Zipcode","Product Card Id","Product Category Id","Product Status","shipping date (DateOrders)"]
df.drop(unwanted_columns, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer State                 180519 non-null  object 
 12  Market                        

In [3]:
#checking for missing values
df2 = df.copy(deep=False)

percent_Miss = (df2.isnull().sum() / len(df2)) * 100
values_missing = df2.isnull().sum()
missingValuesDf = pd.DataFrame({'missing values': values_missing,'percentMissing': percent_Miss})
missingValuesDf

,missing values,percentMissing
Type,0,0.0
Days for shipping (real),0,0.0
Days for shipment (scheduled),0,0.0
Benefit per order,0,0.0
Sales per customer,0,0.0
Delivery Status,0,0.0
Late_delivery_risk,0,0.0
Category Id,0,0.0
Category Name,0,0.0
Customer City,0,0.0


In [4]:
# change object data into int or float

hist_data = df.copy(deep=False)

from sklearn.preprocessing import LabelEncoder
def Change_obj_type(data):
    for column in data.columns:
        if df[column].dtype == type(object):
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
    return df
new_data = Change_obj_type(hist_data)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  int64  
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  int64  
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  int64  
 9   Customer City                  180519 non-null  int64  
 10  Customer Country               180519 non-null  int64  
 11  Customer State                 180519 non-null  int64  
 12  Market                        

# Model Building:-

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# Assuming df is your DataFrame with features and the target variable
# Replace 'Late_delivery_risk' with the actual target variable name if different
X = new_data.drop('Late_delivery_risk', axis=1)
y = new_data['Late_delivery_risk']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train_scaled, y_train)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Select top k features (adjust k as needed)
k = 3
top_k_indices = feature_importances.argsort()[-k:][::-1]

# Select top k features from X
X_train_selected = X_train.iloc[:, top_k_indices]
X_test_selected = X_test.iloc[:, top_k_indices]

# Train the classifier on the selected features
rf_classifier.fit(X_train_selected, y_train)

# Make predictions on the test data with selected features
y_pred_selected = rf_classifier.predict(X_test_selected)

# Display classification report
print('Classification Report:')
print(classification_report(y_test, y_pred_selected))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16392
           1       1.00      1.00      1.00     19712

    accuracy                           1.00     36104
   macro avg       1.00      1.00      1.00     36104
weighted avg       1.00      1.00      1.00     36104

